# Profile NeRF with Timeloop and Accelergy

In [27]:
import json
import numpy as np
import os
import re
import yaml
import traceback

from collections import defaultdict
from profiler import Profiler
from notebook_utils import natural_sort

import sys
sys.path.append("../") # go to parent dir

from accelerating_nerfs.models import VanillaNeRF, patch_forward

## Load NeRF model
We use vanilla NeRFs which are MLPs

In [2]:
# Uncomment to view architecture diagram
# from IPython.display import IFrame
# IFrame("./figures/netdiag-modified.pdf", width=600, height=350)

In [3]:
model = VanillaNeRF()

# Need to patch the forward method for the purpose of mapping to pass in ray directions
# This ensures the bottleneck layer is captured in the timeloop outputs
patch_forward(model)
print(model)

VanillaNeRF(
  (posi_encoder): SinusoidalEncoder()
  (view_encoder): SinusoidalEncoder()
  (mlp): NerfMLP(
    (base): MLP(
      (hidden_activation): ReLU()
      (output_activation): Identity()
      (hidden_layers): ModuleList(
        (0): Linear(in_features=63, out_features=256, bias=True)
        (1): Linear(in_features=256, out_features=256, bias=True)
        (2): Linear(in_features=256, out_features=256, bias=True)
        (3): Linear(in_features=256, out_features=256, bias=True)
        (4): Linear(in_features=256, out_features=256, bias=True)
        (5): Linear(in_features=319, out_features=256, bias=True)
        (6): Linear(in_features=256, out_features=256, bias=True)
        (7): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (sigma_layer): DenseLayer(
      (hidden_activation): ReLU()
      (output_activation): Identity()
      (hidden_layers): ModuleList()
      (output_layer): Linear(in_features=256, out_features=1, bias=True)
    )
    (bottl

/home/workspace/notebooks/../accelerating_nerfs/models.py:254: UserWarning: patched forward of VanillaNeRF to also pass the condition. You should only use this for debugging or with Timeloop and Accelergy
  warnings.warn(


### Configure saving of profiling results
This isn't important so you can ignore the details.

In [4]:
# Accumulate results in this dictionary
profile_results = {}

# Setup saving the profiling results
results_dir = "profile_results"
os.makedirs(results_dir, exist_ok=True)


def save_results():
    all_other_results = {}
    
    for arch, arch_results in profile_results.items():
        # Write the super long results to it's own file
        arch_results_path = os.path.join(results_dir, f"{arch}_results.json")
        with open(arch_results_path, "w") as f:
            json.dump(arch_results["results"], f, indent=4)
            print(f"Saved {arch} results to {arch_results_path}")
        
        # Accumulate the other results as they're shorter and more readable
        other_results = {
            k: v for k, v in arch_results.items()
            if k != "results"
        }
        # Have a pointer to the separate results file
        other_results["results"] = os.path.abspath(arch_results_path)
        all_other_results[arch] = other_results
    
    results_path = os.path.join(results_dir, "results.json")
    with open(results_path, "w") as f:
        json.dump(all_other_results, f, indent=4)

    print(f"Saved profile results to {results_path}")

### Loading NeRF layer shapes
You can ignore this, it's for populating the profiling results with additional debug information.

In [5]:
def nerf_layer_shapes() -> dict:
    """ Load layer shape info from the pytorch2timeloop converter. Returns mapping of layer ID to shape dict """
    nerf_layer_dir = "workloads/nerf"
    keys_should_be_1 = ["Hdilation", "Hstride", "P", "Q", "R", "S", "Wdilation", "Wstride"]
    layer_shapes = {}
    
    for layer_path in natural_sort(os.listdir(nerf_layer_dir)):
        layer_path = os.path.join(nerf_layer_dir, layer_path)
        layer_id = int(layer_path.split("layer")[1].split(".")[0])

        with open(layer_path, "r") as f:
            layer_config = yaml.safe_load(f)

        instance = layer_config['problem']['instance']
        for key in keys_should_be_1:
            assert instance[key] == 1, f"{key} != 1"
            del instance[key]

        # print(f"{os.path.basename(layer_path)}, layer_id={layer_id}, {instance}")
        assert layer_id not in layer_shapes
        layer_shapes[layer_id] = {"shape": instance}
        
    assert layer_shapes, "layer_shapes should not be empty"
    return layer_shapes

### Add densities to NeRF layer problems
Load the layer sparsities from our earlier analysis.

In [65]:
sparsity_results_path = "../accelerating_nerfs/sparsity/2023-05-03_00-21-28_sparsity.json"
with open(sparsity_results_path, "r") as f:
    sparsity_results_dict = json.load(f)

sparsity_results = defaultdict(dict)
sparsity_key = 'avg_sparsity'
sparsity_std_key = 'std_sparsity'

for scene, results in sparsity_results_dict.items():
    if scene == "overall":
        continue
    for layer_name, layer_result in results.items():
        _, num = layer_result["fc_label"].split("_")
        layer_num = int(num) + 1
        # We care about density, which is 1 - sparsity
        sparsity_results[scene][layer_num] = {
            "sparsity": layer_result[sparsity_key],
            "density": 1 - layer_result[sparsity_key]
        }
print(f"Loaded sparsity results for {sparsity_results.keys()}")

# Layer-wise density results
layer_to_densities = defaultdict(list)

for results in sparsity_results.values():
    for layer_name, layer_result in results.items():
        layer_to_densities[layer_name].append(layer_result["density"])

layer_to_avg_density = {layer: np.mean(densities).item() for layer, densities in layer_to_densities.items()}
layer_to_avg_sparsity = {layer: 1 - density for layer, density in layer_to_avg_density.items()}
print("Average sparsity:\n" + "\n".join([f"{k}: {v}" for k, v in layer_to_avg_sparsity.items()]))

Loaded sparsity results for dict_keys(['chair', 'drums', 'ficus', 'hotdog', 'lego', 'materials', 'mic', 'ship'])
Average sparsity:
1: 4.3469441712851165e-08
2: 0.5232157788498581
3: 0.6606506746276802
4: 0.6719614964550142
5: 0.635327070883858
6: 0.49655377627334907
7: 0.5577130213002062
8: 0.6088893305723659
9: 0.5989934303659362
10: 0.5785213852133457
11: 2.1946953143725523e-09
12: 0.6979629018407193


In [75]:
def add_density_to_nerf_layers():
    nerf_layer_dir = "workloads/nerf"

    for layer_path in natural_sort(os.listdir(nerf_layer_dir)):
        layer_path = os.path.join(nerf_layer_dir, layer_path)
        layer_id = int(layer_path.split("layer")[1].split(".")[0])
        print(f"=== For layer {layer_id} at {layer_path} ===")

        with open(layer_path, "r") as f:
            layer_config = yaml.safe_load(f)

        density = layer_to_avg_density[layer_id]
        if density > 0.99:
            print(f"Skipped as density is {density} > 0.99")
            continue

        # Add densities to layer_config
        layer_config['problem']['instance']['densities'] = {
            "Inputs": density,
            "Weights": 1.0,
            # TODO: make this same as next layer density
            "Outputs": density,
        }

        # Write updated layer config
        with open(layer_path, "w") as f:
            yaml.dump(layer_config, f)
        print("Added densities:", layer_config['problem']['instance']['densities'])

## Profile using Timeloop and Accelergy
I think we can safely ignore the 'unknown module type' warnings.

In [76]:
add_density_to_nerf_layers()

=== For layer 1 at workloads/nerf/nerf_layer1.yaml ===
Skipped as density is 0.9999999565305583 > 0.99
=== For layer 2 at workloads/nerf/nerf_layer2.yaml ===
Added densities: {'Inputs': 0.47678422115014185, 'Weights': 1.0, 'Outputs': 0.47678422115014185}
=== For layer 3 at workloads/nerf/nerf_layer3.yaml ===
Added densities: {'Inputs': 0.33934932537231977, 'Weights': 1.0, 'Outputs': 0.33934932537231977}
=== For layer 4 at workloads/nerf/nerf_layer4.yaml ===
Added densities: {'Inputs': 0.3280385035449857, 'Weights': 1.0, 'Outputs': 0.3280385035449857}
=== For layer 5 at workloads/nerf/nerf_layer5.yaml ===
Added densities: {'Inputs': 0.364672929116142, 'Weights': 1.0, 'Outputs': 0.364672929116142}
=== For layer 6 at workloads/nerf/nerf_layer6.yaml ===
Added densities: {'Inputs': 0.5034462237266509, 'Weights': 1.0, 'Outputs': 0.5034462237266509}
=== For layer 7 at workloads/nerf/nerf_layer7.yaml ===
Added densities: {'Inputs': 0.44228697869979383, 'Weights': 1.0, 'Outputs': 0.442286978699

In [77]:
# Don't use simba_like or simple_output_stationary as the mapper constraints are too stringent
# archs = ["eyeriss_like", "simba_like_modified", "simple_output_stationary_modified", "simple_weight_stationary"]
# archs = ["simba_like_modified", "simple_output_stationary_modified"]
archs = ["eyeriss_like_onchip_compression"]
failed_archs = set()

for arch in archs:
    print(20 * '=')
    print(f"Running {arch}")
    print(20 * '=')
    
    # Profile - you should only need to change batch_size if anything
    try:
        profiler = Profiler(
            top_dir='workloads',
            sub_dir='nerf',
            timeloop_dir=f"designs/{arch}",
            arch_name=arch,
            model=model,
            input_size=(1, 3),
            batch_size=128,  # TODO: adjust this, ICARUS uses 128
            convert_fc=True,
            exception_module_names=[]
        )
        add_density_to_nerf_layers()
        results, summary, layer_summary = profiler.profile()
    except Exception as e:
        # TODO: figure this out https://piazza.com/class/ldf2iof72w51sl/post/44
        traceback.print_exc()
        print(f"ERROR: could not run profiler for {arch}, do not trust these results!")
        failed_archs.add(arch)
        continue
    
    # Add nerf layer shapes to the layer summary
    for layer_id in layer_summary:
        layer_summary[layer_id].update(nerf_layer_shapes()[layer_id])
        
    # Print summary information
    for k, v in summary.items():
        print(f"{k}: {v}")
        
    profile_results[arch] = {
        "results": results,
        "summary": summary,
        "layer_summary": layer_summary,
    }
    save_results()

unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.MLP'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.DenseLayer'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.DenseLayer'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.MLP'>
unknown module type <class 'accelerating_nerfs.models.NerfMLP'>
unknown module type <class 'accelerating_nerfs.models.VanillaNeRF'>


Running eyeriss_like_onchip_compression
=== For layer 1 at workloads/nerf/nerf_layer1.yaml ===
Skipped as density is 0.9999999565305583 > 0.99
=== For layer 2 at workloads/nerf/nerf_layer2.yaml ===
Added densities: {'Inputs': 0.47678422115014185, 'Weights': 1.0, 'Outputs': 0.47678422115014185}
=== For layer 3 at workloads/nerf/nerf_layer3.yaml ===
Added densities: {'Inputs': 0.33934932537231977, 'Weights': 1.0, 'Outputs': 0.33934932537231977}
=== For layer 4 at workloads/nerf/nerf_layer4.yaml ===
Added densities: {'Inputs': 0.3280385035449857, 'Weights': 1.0, 'Outputs': 0.3280385035449857}
=== For layer 5 at workloads/nerf/nerf_layer5.yaml ===
Added densities: {'Inputs': 0.364672929116142, 'Weights': 1.0, 'Outputs': 0.364672929116142}
=== For layer 6 at workloads/nerf/nerf_layer6.yaml ===
Added densities: {'Inputs': 0.5034462237266509, 'Weights': 1.0, 'Outputs': 0.5034462237266509}
=== For layer 7 at workloads/nerf/nerf_layer7.yaml ===
Added densities: {'Inputs': 0.44228697869979383, '

running timeloop to get energy and latency...:   0%|          | 0/6 [00:00<?, ?it/s]ERROR: key not found: problem, at line: 0
ERROR: key not found: problem, at line: 0
ERROR: key not found: problem, at line: 0
ERROR: key not found: problem, at line: 0
ERROR: key not found: problem, at line: 0
running timeloop to get energy and latency...: 100%|██████████| 6/6 [00:00<00:00, 42.18it/s]

input file: /home/workspace/notebooks/designs/eyeriss_like_onchip_compression/arch/eyeriss_like_onchip_compression.yaml
input file: /home/workspace/notebooks/designs/eyeriss_like_onchip_compression/arch/components/mac.yaml
input file: /home/workspace/notebooks/designs/eyeriss_like_onchip_compression/arch/components/regfile_metadata.yaml
input file: /home/workspace/notebooks/designs/eyeriss_like_onchip_compression/arch/components/smartbuffer.yaml
input file: /home/workspace/notebooks/designs/eyeriss_like_onchip_compression/arch/components/smartbuffer_metadata.yaml
input file: /home/workspace/notebooks/designs/eyeriss_like_onchip_compression/mapper/mapper.yaml
input file: /home/workspace/notebooks/designs/eyeriss_like_onchip_compression/constraints/eyeriss_like_arch_constraints.yaml
input file: /home/workspace/notebooks/designs/eyeriss_like_onchip_compression/constraints/eyeriss_like_map_constraints.yaml
input file: /home/workspace/notebooks/designs/eyeriss_like_onchip_compression/sparse


Traceback (most recent call last):
  File "/tmp/ipykernel_398/1948455616.py", line 26, in <module>
    results, summary, layer_summary = profiler.profile()
  File "/home/workspace/notebooks/profiler.py", line 358, in profile
    self.populate_profiled_lib(layer_info)
  File "/home/workspace/notebooks/profiler.py", line 306, in populate_profiled_lib
    info = {key: layer_info[layer_id][key] for key in keys_to_include}
  File "/home/workspace/notebooks/profiler.py", line 306, in <dictcomp>
    info = {key: layer_info[layer_id][key] for key in keys_to_include}
KeyError: 'energy'


In [7]:
for arch, arch_results in profile_results.items():
    print(f"===== {arch} =====")
    print(summary)